<a href="https://colab.research.google.com/github/uruenariobo/ai4eng_health_issues/blob/main/02%20-%20Patient%20Survival%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Patient Survival Prediction



1.   Importar dataset

2.   Observar valores null de la informacion


In [1]:
import numpy as np
import pandas as pd
import io
from google.colab import files

import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, 
                            classification_report,
                            roc_auc_score, roc_curve, auc, precision_recall_curve,
                            confusion_matrix)

# from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
import xgboost as xgb
from xgboost import XGBClassifier

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# import shap

pd.set_option('display.max_rows', 250)

In [2]:
! pip install kaggle
! mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!echo '{"username":"gaiaramirezhincapie","key":"b2aa779bb3673bd4974ed05af3a8fb0d"}' > /root/.kaggle/kaggle.json

! kaggle datasets download mitishaagarwal/patient

 60% 5.00M/8.30M [00:00<00:00, 40.4MB/s]
100% 8.30M/8.30M [00:00<00:00, 57.0MB/s]


In [4]:
! unzip patient.zip

Archive:  patient.zip
  inflating: dataset.csv             


# 1. Lectura y preprocesado del Dataset

In [5]:
!kaggle datasets download -d mitishaagarwal/patient

# Function to download and extract files from a zip url
!unzip '/content/ashrae-energy-prediction.zip'

df=pd.read_csv("/content/dataset.csv")

patient.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open /content/ashrae-energy-prediction.zip, /content/ashrae-energy-prediction.zip.zip or /content/ashrae-energy-prediction.zip.ZIP.


In [6]:
df.shape

(91713, 85)

In [7]:
print(df)

       encounter_id  patient_id  hospital_id   age        bmi  \
0             66154       25312          118  68.0  22.730000   
1            114252       59342           81  77.0  27.420000   
2            119783       50777          118  25.0  31.950000   
3             79267       46918          118  81.0  22.640000   
4             92056       34377           33  19.0        NaN   
...             ...         ...          ...   ...        ...   
91708         91592       78108           30  75.0  23.060250   
91709         66119       13486          121  56.0  47.179671   
91710          8981       58179          195  48.0  27.236914   
91711         33776      120598           66   NaN  23.297481   
91712          1671       53612          104  82.0  22.031250   

       elective_surgery  ethnicity gender  height           icu_admit_source  \
0                     0  Caucasian      M   180.3                      Floor   
1                     0  Caucasian      F   160.0          

Se observa la cantidad de valores nulos presentes en el dataset

In [8]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 85 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   encounter_id                   91713 non-null  int64  
 1   patient_id                     91713 non-null  int64  
 2   hospital_id                    91713 non-null  int64  
 3   age                            87485 non-null  float64
 4   bmi                            88284 non-null  float64
 5   elective_surgery               91713 non-null  int64  
 6   ethnicity                      90318 non-null  object 
 7   gender                         91688 non-null  object 
 8   height                         90379 non-null  float64
 9   icu_admit_source               91601 non-null  object 
 10  icu_id                         91713 non-null  int64  
 11  icu_stay_type                  91713 non-null  object 
 12  icu_type                       91713 non-null 

Eliminamos la columna 83 ya que no se encuentra caracterizada

In [9]:
df = df.drop(['Unnamed: 83'], axis = 1)

Eliminamos los datos que no consideramos importantes para el modelo que se desea plantear y analizar

In [10]:
df.drop(['encounter_id','patient_id', 'icu_admit_source','icu_id', 'icu_stay_type', 'icu_type'], axis = 1, inplace = True)

In [11]:
df.drop(['hospital_id','apache_2_diagnosis','apache_3j_diagnosis','apache_4a_hospital_death_prob','apache_4a_icu_death_prob','apache_2_bodysystem','apache_3j_bodysystem'], axis = 1, inplace = True)

In [12]:
df.nunique()

age                               74
bmi                            34888
elective_surgery                   2
ethnicity                          6
gender                             2
height                           401
pre_icu_los_days                9757
weight                          3409
apache_post_operative              2
arf_apache                         2
gcs_eyes_apache                    4
gcs_motor_apache                   6
gcs_unable_apache                  2
gcs_verbal_apache                  5
heart_rate_apache                149
intubated_apache                   2
map_apache                       161
resprate_apache                   74
temp_apache                      191
ventilated_apache                  2
d1_diasbp_max                    120
d1_diasbp_min                     78
d1_diasbp_noninvasive_max        120
d1_diasbp_noninvasive_min         78
d1_heartrate_max                 120
d1_heartrate_min                 154
d1_mbp_max                       125
d

Al observar el dataset es importante notar que los registros no presentan razgos que permitan agruparlos adecuadamente para completar informacion faltante en campos como peso, edad, o estatura. Por esto mismo, es necesario eliminar estos registros para no afectar la construccion del modelo

In [13]:
df.shape

(91713, 71)

In [14]:
df = df[df[['bmi','age', 'weight', 'height']].isna().sum(axis=1) == 0]
df

,age,bmi,elective_surgery,ethnicity,gender,height,pre_icu_los_days,weight,apache_post_operative,arf_apache,...,d1_potassium_min,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,68.0,22.730000,0,Caucasian,M,180.3,0.541667,73.9,0,0.0,...,3.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,77.0,27.420000,0,Caucasian,F,160.0,0.927778,70.2,0,0.0,...,3.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,25.0,31.950000,0,Caucasian,F,172.7,0.000694,95.3,0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,81.0,22.640000,1,Caucasian,F,165.1,0.000694,61.7,1,0.0,...,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,67.0,27.560000,0,Caucasian,M,190.5,0.000694,100.0,0,0.0,...,3.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91706,54.0,19.770448,0,Native American,M,177.8,0.025694,62.5,0,0.0,...,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91708,75.0,23.060250,0,Caucasian,M,177.8,0.298611,72.9,0,0.0,...,4.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
91709,56.0,47.179671,0,Caucasian,F,183.0,0.120139,158.0,0,0.0,...,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91710,48.0,27.236914,0,Caucasian,M,170.2,0.046528,78.9,0,0.0,...,3.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84229 entries, 0 to 91712
Data columns (total 71 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   age                          84229 non-null  float64
 1   bmi                          84229 non-null  float64
 2   elective_surgery             84229 non-null  int64  
 3   ethnicity                    83060 non-null  object 
 4   gender                       84219 non-null  object 
 5   height                       84229 non-null  float64
 6   pre_icu_los_days             84229 non-null  float64
 7   weight                       84229 non-null  float64
 8   apache_post_operative        84229 non-null  int64  
 9   arf_apache                   84229 non-null  float64
 10  gcs_eyes_apache              83149 non-null  float64
 11  gcs_motor_apache             83149 non-null  float64
 12  gcs_unable_apache            83970 non-null  float64
 13  gcs_verbal_apach

In [16]:
df.dropna()

,age,bmi,elective_surgery,ethnicity,gender,height,pre_icu_los_days,weight,apache_post_operative,arf_apache,...,d1_potassium_min,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,68.0,22.730000,0,Caucasian,M,180.3,0.541667,73.9,0,0.0,...,3.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,77.0,27.420000,0,Caucasian,F,160.0,0.927778,70.2,0,0.0,...,3.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
5,67.0,27.560000,0,Caucasian,M,190.5,0.000694,100.0,0,0.0,...,3.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
10,72.0,28.257052,1,Hispanic,F,154.9,0.004861,67.8,1,0.0,...,4.2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
17,46.0,25.845717,0,Hispanic,M,167.6,0.000000,72.6,0,0.0,...,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91698,47.0,51.439842,1,Caucasian,M,195.0,0.033333,186.0,1,0.0,...,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91706,54.0,19.770448,0,Native American,M,177.8,0.025694,62.5,0,0.0,...,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91708,75.0,23.060250,0,Caucasian,M,177.8,0.298611,72.9,0,0.0,...,4.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
91709,56.0,47.179671,0,Caucasian,F,183.0,0.120139,158.0,0,0.0,...,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [17]:
df.dropna().info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62498 entries, 0 to 91712
Data columns (total 71 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   age                          62498 non-null  float64
 1   bmi                          62498 non-null  float64
 2   elective_surgery             62498 non-null  int64  
 3   ethnicity                    62498 non-null  object 
 4   gender                       62498 non-null  object 
 5   height                       62498 non-null  float64
 6   pre_icu_los_days             62498 non-null  float64
 7   weight                       62498 non-null  float64
 8   apache_post_operative        62498 non-null  int64  
 9   arf_apache                   62498 non-null  float64
 10  gcs_eyes_apache              62498 non-null  float64
 11  gcs_motor_apache             62498 non-null  float64
 12  gcs_unable_apache            62498 non-null  float64
 13  gcs_verbal_apach

# Modelo supervisado



In [18]:
numerical_cat = [
 'elective_surgery',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis']

categorical = ['ethnicity',
 'gender']

icu_data = pd.get_dummies(df,
    prefix='isin',
    prefix_sep='_',
    columns=categorical,
    drop_first=False)
icu_data.reset_index(drop = True, inplace = True)
icu_data

X = icu_data.drop(['hospital_death'], axis=1)
y = icu_data['hospital_death']

X_new = X[['age','d1_spo2_min','d1_resprate_max','d1_heartrate_min']]

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.30, 
                                                    random_state=11, 
                                                    stratify = y)

y_train.value_counts()
y_test.value_counts()

0    23186
1     2083
Name: hospital_death, dtype: int64

#Modelado de datos

In [19]:
import plotly.graph_objs as go
import plotly.subplots as tls
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
import plotly.offline as py

def plot_model_performance(trained_model, true_labels, predicted_labels):
        conf_matrix = confusion_matrix(y_test, y_hat)
        trace1 = go.Heatmap(z = conf_matrix  ,x = ["0 (pred)","1 (pred)"],
                            y = ["0 (true)","1 (true)"],xgap = 2, ygap = 2, 
                            colorscale = 'Viridis', showscale  = False)

        #Show metrics
        tp = conf_matrix[1,1]
        fn = conf_matrix[1,0]
        fp = conf_matrix[0,1]
        tn = conf_matrix[0,0]
        Accuracy  =  ((tp+tn)/(tp+tn+fp+fn))
        Precision =  (tp/(tp+fp))
        Recall    =  (tp/(tp+fn))
        F1_score  =  (2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn)))))

        show_metrics = pd.DataFrame(data=[[Accuracy , Precision, Recall, F1_score]])
        show_metrics = show_metrics.T

        colors = ['gold', 'lightgreen', 'lightcoral', 'lightskyblue']
        trace2 = go.Bar(x = (show_metrics[0].values), 
                      y = ['Accuracy', 'Precision', 'Recall', 'F1_score'], text = np.round_(show_metrics[0].values,4),
                        textposition = 'auto',
                      orientation = 'h', opacity = 0.8,marker=dict(
                color=colors,
                line=dict(color='#000000',width=1.5)))
        
        #Roc curve
        model_roc_auc = round(roc_auc_score(y_test, y_hat) , 3)
        fpr, tpr, t = roc_curve(y_test, y_hat)
        trace3 = go.Scatter(x = fpr,y = tpr,
                            name = "Roc : " + str(model_roc_auc),
                            line = dict(color = ('rgb(22, 96, 167)'),width = 2), fill='tozeroy')
        trace4 = go.Scatter(x = [0,1],y = [0,1],
                            line = dict(color = ('black'),width = 1.5,
                            dash = 'dot'))
        
        # Precision-recall curve
        precision, recall, thresholds = precision_recall_curve(y_test, y_hat)
        trace5 = go.Scatter(x = recall, y = precision,
                            name = "Precision" + str(precision),
                            line = dict(color = ('lightcoral'),width = 2), fill='tozeroy')
        
        
        #plots
        trained_model = trained_model
        
        #Subplots
        fig = tls.make_subplots(rows=2, cols=2, print_grid=False, 
                              specs=[ 
    #                               [{'colspan': 2}, None],
                                    [{}, {}],
                                    [{}, {}],
                                    
    #                                  [{'colspan': 2}, None]
                                    ],
                              subplot_titles=('Confusion Matrix',
                                            'Metrics',
                                            'ROC curve'+" "+ '('+ str(model_roc_auc)+')',
                                            'Precision - Recall curve',
                                            ))
        
        fig.append_trace(trace1,1,1)
        fig.append_trace(trace2,1,2)
        fig.append_trace(trace3,2,1)
        fig.append_trace(trace4,2,1)
        fig.append_trace(trace5,2,2)
        
        fig['layout'].update(showlegend = False, title = '<b>Model performance report</b><br>'+str(trained_model),
                            autosize = False, height = 1500,width = 830,
                            plot_bgcolor = 'rgba(240,240,240, 0.95)',
                            paper_bgcolor = 'rgba(240,240,240, 0.95)',
                            margin = dict(b = 195))
        fig.layout.titlefont.size = 14
        
        py.iplot(fig)

In [20]:
print("Valores NaN en y_test:", y_test.isna().sum())

Valores NaN en y_test: 0


#Definición de parámetros

In [21]:
# Importar la clase de KFold y GridSearchCV de scikit-learn
from sklearn.model_selection import KFold, GridSearchCV

# Importar la clase de XGBClassifier de xgboost
from xgboost import XGBClassifier

# Configurar el generador de índices para la validación cruzada
kf = KFold(n_splits=3, shuffle=True, random_state=42).split(X=X_train, y=y_train)

# Definir los parámetros para ajustar el modelo de XGBoost
params = {
    'booster': ["gbtree"],
    'learning_rate': [0.1],
    'n_estimators': range(100, 500, 100),
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'max_depth': [5],
    "scale_pos_weight": [1]
}

# Definir los parámetros para el ajuste temprano y la evaluación
fit_params_of_xgb = {
    "early_stopping_rounds": 100, 
    "eval_metric" : 'auc', 
    "eval_set" : [(X_test, y_test)],
    'verbose': 100,
}

# Crear una instancia del clasificador XGBoost
xgb_estimator = XGBClassifier(objective='binary:logistic')

# Realizar la búsqueda de la mejor combinación de parámetros usando GridSearchCV
gsearch = GridSearchCV(
    estimator=xgb_estimator,
    param_grid=params,
    scoring='roc_auc',
    n_jobs=-1,
    cv=kf
)

# Entrenar el modelo usando GridSearchCV y los parámetros definidos anteriormente
xgb_model = gsearch.fit(X=X_train, y=y_train, **fit_params_of_xgb)

# Imprimir los mejores parámetros y el puntaje obtenido en la búsqueda de hiperparámetros
print(gsearch.best_params_, gsearch.best_score_)


[0]	validation_0-auc:0.69568
[99]	validation_0-auc:0.76184
{'booster': 'gbtree', 'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 0.8} 0.7444847779033812


#Evaluación preliminar del Modelo

In [22]:
# Crear el modelo XGBoost sintonizado
xgb_tuned = XGBClassifier(
    n_estimators=3000,
    objective='binary:logistic',
    booster="gbtree",
    learning_rate=0.01,
    scale_pos_weight=1,
    max_depth=4,
    min_child_weight=6,
    gamma=0,
    subsample=0.4,
    colsample_bytree=0.8,
    reg_alpha=0.08,
    n_jobs=-1
)

# Entrenar el modelo en los datos de entrenamiento
xgb_tuned.fit(X_train.select_dtypes(include='number'), y_train.values.ravel())

# Realizar predicciones en los datos de prueba
y_hat = xgb_tuned.predict(X_test.select_dtypes(include='number'))

# Imprimir las predicciones de las filas 21 a 30
print(y_hat[21:30])

# Evaluar el rendimiento del modelo
plot_model_performance(xgb_tuned,y_test, y_hat)

# Tomar una muestra aleatoria de 2500 filas de los datos de prueba
X_test_sample = X_test.sample(n=2500, random_state=42)


[0 0 0 0 0 0 0 0 0]
